## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
df = sqlContext.read.load('/FileStore/tables/daily_weather.csv',format='com.databricks.spark.csv',header='true',inferSchema='true')

In [4]:
df.describe().show()

+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
summary| number| air_pressure_9am| air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am| rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
 count| 1095| 1092| 1090| 1091| 1092| 1092| 1091| 1089| 1092| 1095| 1095|
 mean| 547.0|918.8825513138094| 64.93300141287072| 142.2355107005759| 5.50828424225493| 148.95351796516923| 7.019513529175272| 0.20307895225211126| 294.1080522756142| 34.24140205923536| 35.34472714825898|
 stddev|316.24357700987383|3.184161180386833|11.175514003175877| 69.13785928889189|4.5528134655317185| 67.23801294602953| 5.598209170780958| 1.5939521253574893|1598.0787786601481| 25.472066802250055| 22.524079453587273|
 min| 0|907.9900000000024|36.752000000000685| 15.500000000000046| 0.69345139999974| 28.89999999999991|1.1855782000000479| 0.0| 0.0| 6.090000000001012| 5.3000000000006855|
 max| 1094|929.3200000000012| 98.90599999999992| 343.4|23.554978199999763| 312.19999999999993| 29.84077959999996| 24.01999999999907| 17704.0| 92.6200000000002| 92.2500000000003|
+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+

In [5]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
summary| air_temp_9am|
+-------+------------------+
 count| 1090|
 mean| 64.93300141287072|
 stddev|11.175514003175877|
 min|36.752000000000685|
 max| 98.90599999999992|
+-------+------------------+

In [6]:
df.describe().toPandas().transpose()

Out[ 6 ]: 
 0 1 2 \
summary count mean stddev 
number 1095 547.0 316.24357700987383 
air_pressure_9am 1092 918.8825513138094 3.184161180386833 
air_temp_9am 1090 64.93300141287072 11.175514003175877 
avg_wind_direction_9am 1091 142.2355107005759 69.13785928889189 
avg_wind_speed_9am 1092 5.50828424225493 4.5528134655317185 
max_wind_direction_9am 1092 148.95351796516923 67.23801294602953 
max_wind_speed_9am 1091 7.019513529175272 5.598209170780958 
rain_accumulation_9am 1089 0.20307895225211126 1.5939521253574893 
rain_duration_9am 1092 294.1080522756142 1598.0787786601481 
relative_humidity_9am 1095 34.24140205923536 25.472066802250055 
relative_humidity_3pm 1095 35.34472714825898 22.524079453587273 

 3 4 
summary min max 
number 0 1094 
air_pressure_9am 907.9900000000024 929.3200000000012 
air_temp_9am 36.752000000000685 98.90599999999992 
avg_wind_direction_9am 15.500000000000046 343.4 
avg_wind_speed_9am 0.69345139999974 23.554978199999763 
max_wind_direction_9am 28.89999999999991 312.19999999999993 
max_wind_speed_9am 1.1855782000000479 29.84077959999996 
rain_accumulation_9am 0.0 24.01999999999907 
rain_duration_9am 0.0 17704.0 
relative_humidity_9am 6.090000000001012 92.6200000000002 
relative_humidity_3pm 5.3000000000006855 92.2500000000003

In [7]:
sc

Out[ 7 ]: <SparkContext master=local[8] appName=Databricks Shell>

In [8]:
from pyspark.sql.functions import avg
imputeDF = df
removeAllDF = df.na.drop()

In [9]:
for x in imputeDF.columns :
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x , meanValue)
    imputeDF = imputeDF.na.fill(meanValue,[x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533

In [10]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
summary| air_temp_9am|
+-------+------------------+
 count| 1090|
 mean| 64.93300141287072|
 stddev|11.175514003175877|
 min|36.752000000000685|
 max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
summary| air_temp_9am|
+-------+------------------+
 count| 1095|
 mean| 64.93341058219818|
 stddev| 11.14994819992023|
 min|36.752000000000685|
 max| 98.90599999999992|
+-------+------------------+

In [11]:
 tt = removeAllDF.filter(removeAllDF.air_pressure_9am > 911.736).filter(removeAllDF.air_pressure_9am < 914.67).collect()

In [12]:
len(tt)

Out[ 12 ]: 77